In [61]:
from utils import read_images, save_min_projection_imgs
from skimage import io, color
import numpy as np
import napari
import pyclesperanto_prototype as cle
from skimage import morphology
from skimage import io, morphology
from skimage.color import rgb2gray
import numpy as np

# Select a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<Intel(R) UHD Graphics 630 on Platform: Intel(R) OpenCL HD Graphics (2 refs)>

In [62]:
images_per_well = read_images("./data/COLOR_4x_optimal_DoF.2024-02-13-15-22-50")

In [63]:
save_min_projection_imgs(images_per_well)

In [64]:
# Load the image
image_path = './output/MIN_projections/A01.tif'
image = io.imread(image_path)

In [65]:
# Load the image
image_path = './output/MIN_projections/A01.tif'
image = io.imread(image_path)

# Convert to grayscale
image_gray = color.rgb2gray(image)

# Rescale to 0-255 and convert to uint8
image_gray_uint8 = (image_gray * 255).astype(np.uint8)

# Tophat filtering
result_image = None
radius = 20
inverted_image = 255 - image_gray_uint8
img_gpu = cle.push(inverted_image)

result_image = cle.top_hat_box(img_gpu, result_image, radius_x=radius, radius_y=radius)
result_image_cle = cle.pull(result_image)

square_kernel = morphology.square(radius)
result_image_scikit = morphology.white_tophat(inverted_image, footprint=square_kernel)

In [66]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(image, name=f"input")
viewer.add_image(image_gray, name=f"gray_float")
viewer.add_image(image_gray_uint8, name=f"gray_uint8")
viewer.add_image(result_image_cle, name=f"cle-box")
viewer.add_image(result_image_scikit, name=f"scikit{radius}")

<Image layer 'scikit20' at 0x1922cb8faf0>